In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf

import math
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import gzip
import tensorflow as tf
# tf.random.set_seed(1004)
tf.random.set_seed(2)

# Data Load

In [2]:
label = np.load("./data/2030days_label.npy")
data_10days = np.load("./data/10days_data.npy")
data_20days = np.load("./data/20days_data.npy")
data_30days = np.load("./data/30days_data.npy")

label.shape, data_10days.shape, data_20days.shape, data_30days.shape

((7799,), (7799, 10, 4069), (7799, 20, 4069), (7799, 30, 4069))

In [3]:
def MAE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.abs(y_true-y_pred)))

def MSE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.square(y_true-y_pred)))

def RMSE(y_true, y_pred):
    return "{:.2e}".format(np.sqrt(np.mean(np.square(y_true-y_pred))))

In [4]:
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier
from keras.layers import InputLayer, GRU

In [5]:
# 사용할 data 지정
data = data_10days

#train, test split
i = round(data.shape[0] * 0.8)
X_train = data[:i,:,:]
y_train = label[:i]


X_test = data[i:,:,:]
y_test = label[i:]


X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6239, 10, 4069), (6239,), (1560, 10, 4069), (1560,))

# Modeling

- seed(1004) : 73.3333333
- seed(5555) : 73.1

- tf.random.set_seed(1) : 정확도 : 0.7403846153846154
- tf.random.set_seed(10) : 

In [6]:
%%time
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32, return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           2149376   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,211,233
Trainable params: 2,211,233
Non-trainable params: 0
_________________________________________________________________
[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb9850db7c0>), ('model2', <keras.wrappers.

25/25 [==============================] - 5s 213ms/step - loss: 0.0118 - acc: 0.9928
Epoch 12/20
25/25 [==============================] - 5s 214ms/step - loss: 0.0114 - acc: 0.9926
Epoch 13/20
25/25 [==============================] - 5s 210ms/step - loss: 0.0118 - acc: 0.9925
Epoch 14/20
25/25 [==============================] - 5s 213ms/step - loss: 0.0111 - acc: 0.9921
Epoch 15/20
25/25 [==============================] - 5s 201ms/step - loss: 0.0102 - acc: 0.9937
Epoch 16/20
25/25 [==============================] - 5s 208ms/step - loss: 0.0103 - acc: 0.9941
Epoch 17/20
25/25 [==============================] - 5s 194ms/step - loss: 0.0103 - acc: 0.9936
Epoch 18/20
25/25 [==============================] - 5s 208ms/step - loss: 0.0103 - acc: 0.9931
Epoch 19/20
25/25 [==============================] - 5s 208ms/step - loss: 0.0100 - acc: 0.9937
Epoch 20/20
25/25 [==============================] - 5s 210ms/step - loss: 0.0104 - acc: 0.9937
Epoch 1/20
25/25 [==============================] - 

25/25 [==============================] - 6s 244ms/step - loss: 0.0057 - acc: 0.9963
Epoch 17/20
25/25 [==============================] - 6s 247ms/step - loss: 0.0057 - acc: 0.9965
Epoch 18/20
25/25 [==============================] - 6s 249ms/step - loss: 0.0057 - acc: 0.9965
Epoch 19/20
25/25 [==============================] - 6s 241ms/step - loss: 0.0052 - acc: 0.9965
Epoch 20/20
25/25 [==============================] - 6s 238ms/step - loss: 0.0053 - acc: 0.9968
Epoch 1/20
25/25 [==============================] - 7s 250ms/step - loss: 0.0054 - acc: 0.9968
Epoch 2/20
25/25 [==============================] - 6s 246ms/step - loss: 0.0052 - acc: 0.9968
Epoch 3/20
25/25 [==============================] - 6s 241ms/step - loss: 0.0053 - acc: 0.9968
Epoch 4/20
25/25 [==============================] - 5s 192ms/step - loss: 0.0054 - acc: 0.9968
Epoch 5/20
25/25 [==============================] - 6s 231ms/step - loss: 0.0052 - acc: 0.9968
Epoch 6/20
25/25 [==============================] - 6s 23

25/25 [==============================] - 6s 248ms/step - loss: 0.0035 - acc: 0.9981
Epoch 2/20
25/25 [==============================] - 6s 234ms/step - loss: 0.0035 - acc: 0.9981
Epoch 3/20
25/25 [==============================] - 6s 253ms/step - loss: 0.0035 - acc: 0.9981
Epoch 4/20
25/25 [==============================] - 6s 231ms/step - loss: 0.0035 - acc: 0.9981
Epoch 5/20
25/25 [==============================] - 6s 245ms/step - loss: 0.0034 - acc: 0.9981
Epoch 6/20
25/25 [==============================] - 6s 245ms/step - loss: 0.0035 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 6s 242ms/step - loss: 0.0035 - acc: 0.9981
Epoch 8/20
25/25 [==============================] - 6s 250ms/step - loss: 0.0034 - acc: 0.9981
Epoch 9/20
25/25 [==============================] - 6s 248ms/step - loss: 0.0035 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 6s 242ms/step - loss: 0.0036 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 6s 249m

25/25 [==============================] - 6s 232ms/step - loss: 0.0034 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 6s 236ms/step - loss: 0.0034 - acc: 0.9981
Epoch 8/20
25/25 [==============================] - 6s 235ms/step - loss: 0.0034 - acc: 0.9981
Epoch 9/20
25/25 [==============================] - 6s 240ms/step - loss: 0.0034 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 6s 232ms/step - loss: 0.0035 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 6s 238ms/step - loss: 0.0034 - acc: 0.9981
Epoch 12/20
25/25 [==============================] - 6s 229ms/step - loss: 0.0034 - acc: 0.9981
Epoch 13/20
25/25 [==============================] - 6s 238ms/step - loss: 0.0035 - acc: 0.9981
Epoch 14/20
25/25 [==============================] - 6s 231ms/step - loss: 0.0034 - acc: 0.9981
Epoch 15/20
25/25 [==============================] - 4s 172ms/step - loss: 0.0034 - acc: 0.9981
Epoch 16/20
25/25 [==============================] - 4s

...

25/25 [==============================] - 6s 245ms/step - loss: 0.0034 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 6s 247ms/step - loss: 0.0034 - acc: 0.9981
Epoch 8/20
25/25 [==============================] - 6s 251ms/step - loss: 0.0034 - acc: 0.9981
Epoch 9/20
25/25 [==============================] - 6s 219ms/step - loss: 0.0034 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 5s 210ms/step - loss: 0.0034 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 5s 193ms/step - loss: 0.0034 - acc: 0.9981
Epoch 12/20
25/25 [==============================] - 5s 218ms/step - loss: 0.0034 - acc: 0.9981
Epoch 13/20
25/25 [==============================] - 5s 217ms/step - loss: 0.0034 - acc: 0.9981
Epoch 14/20
25/25 [==============================] - 6s 253ms/step - loss: 0.0033 - acc: 0.9981
Epoch 15/20
25/25 [==============================] - 6s 251ms/step - loss: 0.0034 - acc: 0.9981
Epoch 16/20
25/25 [==============================] - 6s

25/25 [==============================] - 6s 253ms/step - loss: 0.0034 - acc: 0.9979
Epoch 12/20
25/25 [==============================] - 6s 244ms/step - loss: 0.0034 - acc: 0.9979
Epoch 13/20
25/25 [==============================] - 6s 244ms/step - loss: 0.0035 - acc: 0.9979
Epoch 14/20
25/25 [==============================] - 6s 247ms/step - loss: 0.0035 - acc: 0.9979
Epoch 15/20
25/25 [==============================] - 6s 248ms/step - loss: 0.0034 - acc: 0.9979
Epoch 16/20
25/25 [==============================] - 6s 221ms/step - loss: 0.0035 - acc: 0.9979
Epoch 17/20
25/25 [==============================] - 6s 241ms/step - loss: 0.0036 - acc: 0.9979
Epoch 18/20
25/25 [==============================] - 6s 239ms/step - loss: 0.0034 - acc: 0.9979
Epoch 19/20
25/25 [==============================] - 6s 239ms/step - loss: 0.0035 - acc: 0.9979
Epoch 20/20
25/25 [==============================] - 6s 236ms/step - loss: 0.0035 - acc: 0.9979
Epoch 1/20
25/25 [==============================] - 

25/25 [==============================] - 6s 249ms/step - loss: 0.0036 - acc: 0.9979
Epoch 17/20
25/25 [==============================] - 6s 241ms/step - loss: 0.0036 - acc: 0.9979
Epoch 18/20
25/25 [==============================] - 6s 240ms/step - loss: 0.0035 - acc: 0.9979
Epoch 19/20
25/25 [==============================] - 6s 238ms/step - loss: 0.0037 - acc: 0.9979
Epoch 20/20
25/25 [==============================] - 5s 220ms/step - loss: 0.0035 - acc: 0.9979
Epoch 1/20
25/25 [==============================] - 6s 245ms/step - loss: 0.0035 - acc: 0.9979
Epoch 2/20
25/25 [==============================] - 6s 247ms/step - loss: 0.0035 - acc: 0.9979
Epoch 3/20
25/25 [==============================] - 6s 240ms/step - loss: 0.0036 - acc: 0.9979
Epoch 4/20
25/25 [==============================] - 6s 223ms/step - loss: 0.0035 - acc: 0.9979
Epoch 5/20
25/25 [==============================] - 6s 245ms/step - loss: 0.0034 - acc: 0.9979
Epoch 6/20
25/25 [==============================] - 6s 23

25/25 [==============================] - 5s 202ms/step - loss: 0.0033 - acc: 0.9981
Epoch 2/20
25/25 [==============================] - 5s 205ms/step - loss: 0.0033 - acc: 0.9981
Epoch 3/20
25/25 [==============================] - 5s 206ms/step - loss: 0.0034 - acc: 0.9981
Epoch 4/20
25/25 [==============================] - 5s 198ms/step - loss: 0.0033 - acc: 0.9981
Epoch 5/20
25/25 [==============================] - 6s 222ms/step - loss: 0.0033 - acc: 0.9981
Epoch 6/20
25/25 [==============================] - 6s 242ms/step - loss: 0.0033 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 6s 236ms/step - loss: 0.0033 - acc: 0.9981
Epoch 8/20
25/25 [==============================] - 6s 251ms/step - loss: 0.0034 - acc: 0.9981
Epoch 9/20
25/25 [==============================] - 6s 242ms/step - loss: 0.0034 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 6s 246ms/step - loss: 0.0034 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 6s 240m

25/25 [==============================] - 5s 213ms/step - loss: 0.0033 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 5s 195ms/step - loss: 0.0034 - acc: 0.9981
Epoch 8/20
25/25 [==============================] - 5s 221ms/step - loss: 0.0034 - acc: 0.9981
Epoch 9/20
25/25 [==============================] - 5s 217ms/step - loss: 0.0034 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 5s 209ms/step - loss: 0.0033 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 5s 208ms/step - loss: 0.0034 - acc: 0.9981
Epoch 12/20
25/25 [==============================] - 5s 211ms/step - loss: 0.0034 - acc: 0.9981
Epoch 13/20
25/25 [==============================] - 5s 211ms/step - loss: 0.0034 - acc: 0.9981
Epoch 14/20
25/25 [==============================] - 5s 208ms/step - loss: 0.0032 - acc: 0.9981
Epoch 15/20
25/25 [==============================] - 5s 211ms/step - loss: 0.0034 - acc: 0.9981
Epoch 16/20
25/25 [==============================] - 5s

In [7]:
print("MSE : ", MSE(y_test, preds))

MSE :  2.51e-01


In [10]:
import random
_list = list(range(1560))
random.shuffle(_list)
patient = {}
for i in range(1560):
    patient[i] = _list[i]

In [15]:
import time
from sklearn import metrics 
def item_acc(num):
    df = X_test.copy()
    for i in range(1560):
        df[i][:,num] = df[patient[i]][:,num]

    preds = final_model.predict(df)
    preds[preds>0.5]=1
    preds[preds<=0.5]=0
    acc = metrics.accuracy_score(y_test, preds)
    loss = MSE(y_test, preds)
    print(f'item idx {num} acc & loss :', acc, loss)
    return [acc, loss]

In [16]:
item_acc(1)

item idx 1 acc & loss : 0.7487179487179487 2.51e-01


[0.7487179487179487, '2.51e-01']

In [17]:
idx = [182, 186, 206, 1995,41,119,621,94,121,56]

In [18]:
%%time
result ={}
loss = {}
for i in idx:    
    temp = item_acc(i)
    result[i] = temp[0]
    loss[i] = temp[1]

item idx 182 acc & loss : 0.7506410256410256 2.49e-01
item idx 186 acc & loss : 0.7448717948717949 2.55e-01
item idx 206 acc & loss : 0.7410256410256411 2.59e-01
item idx 1995 acc & loss : 0.7384615384615385 2.62e-01
item idx 41 acc & loss : 0.7371794871794872 2.63e-01
item idx 119 acc & loss : 0.7423076923076923 2.58e-01
item idx 621 acc & loss : 0.7474358974358974 2.53e-01
item idx 94 acc & loss : 0.7339743589743589 2.66e-01
item idx 121 acc & loss : 0.7397435897435898 2.60e-01
item idx 56 acc & loss : 0.7423076923076923 2.58e-01
CPU times: user 2h 23min 8s, sys: 7min 55s, total: 2h 31min 3s
Wall time: 3min 30s


In [19]:
idx2 = [ 206,  121,   56,   20,  103,   34,  119,  184,  203,  204,   26,
               8,   48,   95, 3833,  187,   41, 2612,  186,  278,  197,  193,
               5,   94,   82,   19, 2196,    6, 2213,  305,   54, 2634,   86,
              21,   32, 3046,  188,  284,  861,  217,    2, 3457,  845,  124,
              11,  307,  619,    7, 2244,  191,   10, 1979,  182,   12, 1995,
            2190, 2050, 1993, 2149, 1994, 2453, 2488, 3722,   13, 2027, 3114,
            3104, 1828,  155, 2003,  162,   83, 2341, 1583, 2389, 1081,  621,
            3554, 2022, 1998,  383,   63, 2222, 3034, 1904,   81, 2327,   71,
            1981,    0, 2627, 3469,   25, 2291, 2107, 1000, 3219, 3899,  860,
            1034]

In [20]:
%%time
result ={}
loss = {}
for i in idx2:    
    temp = item_acc(i)
    result[i] = temp[0]
    loss[i] = temp[1]

item idx 206 acc & loss : 0.7410256410256411 2.59e-01
item idx 121 acc & loss : 0.7397435897435898 2.60e-01
item idx 56 acc & loss : 0.7423076923076923 2.58e-01
item idx 20 acc & loss : 0.7455128205128205 2.54e-01
item idx 103 acc & loss : 0.7474358974358974 2.53e-01
item idx 34 acc & loss : 0.7423076923076923 2.58e-01
item idx 119 acc & loss : 0.7423076923076923 2.58e-01
item idx 184 acc & loss : 0.7416666666666667 2.58e-01
item idx 203 acc & loss : 0.7391025641025641 2.61e-01
item idx 204 acc & loss : 0.7455128205128205 2.54e-01
item idx 26 acc & loss : 0.7493589743589744 2.51e-01
item idx 8 acc & loss : 0.7467948717948718 2.53e-01
item idx 48 acc & loss : 0.7378205128205129 2.62e-01
item idx 95 acc & loss : 0.7442307692307693 2.56e-01
item idx 3833 acc & loss : 0.7314102564102564 2.69e-01
item idx 187 acc & loss : 0.7403846153846154 2.60e-01
item idx 41 acc & loss : 0.7371794871794872 2.63e-01
item idx 2612 acc & loss : 0.7480769230769231 2.52e-01
item idx 186 acc & loss : 0.7448717

In [25]:
dict(sorted(loss.items(), key=lambda item: item[1], reverse=True))

{217: '2.73e-01',
 3833: '2.69e-01',
 94: '2.66e-01',
 41: '2.63e-01',
 48: '2.62e-01',
 1995: '2.62e-01',
 203: '2.61e-01',
 121: '2.60e-01',
 187: '2.60e-01',
 206: '2.59e-01',
 56: '2.58e-01',
 34: '2.58e-01',
 119: '2.58e-01',
 184: '2.58e-01',
 2341: '2.58e-01',
 1583: '2.58e-01',
 5: '2.57e-01',
 2196: '2.57e-01',
 2213: '2.57e-01',
 3046: '2.57e-01',
 7: '2.57e-01',
 2149: '2.57e-01',
 1998: '2.57e-01',
 95: '2.56e-01',
 619: '2.56e-01',
 71: '2.56e-01',
 0: '2.56e-01',
 2627: '2.56e-01',
 186: '2.55e-01',
 124: '2.55e-01',
 1979: '2.55e-01',
 1828: '2.55e-01',
 2327: '2.55e-01',
 1000: '2.55e-01',
 20: '2.54e-01',
 204: '2.54e-01',
 305: '2.54e-01',
 10: '2.54e-01',
 2050: '2.54e-01',
 1993: '2.54e-01',
 2488: '2.54e-01',
 2027: '2.54e-01',
 2022: '2.54e-01',
 383: '2.54e-01',
 1904: '2.54e-01',
 1981: '2.54e-01',
 3899: '2.54e-01',
 1034: '2.54e-01',
 103: '2.53e-01',
 8: '2.53e-01',
 197: '2.53e-01',
 193: '2.53e-01',
 19: '2.53e-01',
 54: '2.53e-01',
 21: '2.53e-01',
 188: '

In [26]:
ilove = pd.read_csv('./data/data2d_forposter.csv')
data = pd.read_csv('./data/merge_data.csv')

In [37]:
items = [217, 3833, 94, 41, 48, 1995, 203, 121, 187, 206, 56, 34]

In [38]:
for i in items:
    print(ilove.columns[i])

51301
63323026201
50970
50893
50902
338004904
51274
51006
51249
51277
50912
50882


In [31]:
real_item = [51301,63323026201,50970,50893,50902,338004904,51274,51006,51249,51277,50912]

In [32]:
data[data['ITEM'].isin(real_item)]

,SUBJECT_ID,DATE,ITEM,Type
39,9,2149-11-10,50912,LAB
50,9,2149-11-10,51301,LAB
51,9,2149-11-10,51301,LAB
55,9,2149-11-10,51274,LAB
93,9,2149-11-11,50912,LAB
...,...,...,...,...
9221871,99985,2181-02-12,63323026201,PRE
9221893,99985,2181-02-16,51277,LAB
9221904,99985,2181-02-22,51277,LAB
9221913,99985,2181-03-06,51277,LAB


In [33]:
lab_d = pd.read_csv('/data/MIMIC_III/D_LABITEMS.csv')

In [39]:
real_item1 = [51301,50970,50893,50902,51274,51006,51249,51277,50912, 50882]

In [40]:
lab_d[lab_d['ITEMID'].isin(real_item1)]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
209,83,50882,Bicarbonate,Blood,Chemistry,1963-8
220,94,50893,"Calcium, Total",Blood,Chemistry,2000-8
229,103,50902,Chloride,Blood,Chemistry,2075-0
239,113,50912,Creatinine,Blood,Chemistry,2160-0
297,171,50970,Phosphate,Blood,Chemistry,2777-1
332,206,51006,Urea Nitrogen,Blood,Chemistry,3094-0
575,449,51249,MCHC,Blood,Hematology,786-4
600,474,51274,PT,Blood,Hematology,5902-2
603,477,51277,RDW,Blood,Hematology,788-0
627,501,51301,White Blood Cells,Blood,Hematology,804-5


In [41]:
mse = 2.51e-01

In [42]:
item_mse = [2.73e-01,2.66e-01,2.63e-01,2.62e-01,2.61e-01,2.60e-01, 2.60e-01,2.59e-01,2.58e-01,2.58e-01]

In [46]:
for i in item_mse:
    print(mse - i)

-0.02200000000000002
-0.015000000000000013
-0.01200000000000001
-0.01100000000000001
-0.010000000000000009
-0.009000000000000008
-0.009000000000000008
-0.008000000000000007
-0.007000000000000006
-0.007000000000000006


In [48]:
%%time
model1 = Sequential()
model1.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model1.add(LSTM(units=128, return_sequences=False))
# model1.add(LSTM(units=64, return_sequences=True))
# model1.add(LSTM(units=32, return_sequences=False))
# model1.add(LSTM(units=32))
model1.add(Dropout(0.2))
model1.add(Dense(units=1))
model1.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model1.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model1, epochs=50, batch_size=1024)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model1 = VotingClassifier(estimators = estimator, voting = 'soft')
final_model1.fit(X_train,y_train)

preds = final_model1.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               2149376   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 2,149,505
Trainable params: 2,149,505
Non-trainable params: 0
_________________________________________________________________
[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb88443f850>), ('model2', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb8942e9370>), ('model3', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb8846ba280>), ('model4', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb8845fb910>), ('model5', <keras.wrappers.scikit_lea

7/7 [==============================] - 1s 122ms/step - loss: 0.0330 - acc: 0.9746
Epoch 36/50
7/7 [==============================] - 1s 123ms/step - loss: 0.0334 - acc: 0.9764
Epoch 37/50
7/7 [==============================] - 1s 120ms/step - loss: 0.0300 - acc: 0.9768
Epoch 38/50
7/7 [==============================] - 1s 127ms/step - loss: 0.0289 - acc: 0.9786
Epoch 39/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0290 - acc: 0.9765
Epoch 40/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0280 - acc: 0.9767
Epoch 41/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0275 - acc: 0.9782
Epoch 42/50
7/7 [==============================] - 1s 121ms/step - loss: 0.0264 - acc: 0.9798
Epoch 43/50
7/7 [==============================] - 1s 127ms/step - loss: 0.0271 - acc: 0.9771
Epoch 44/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0254 - acc: 0.9817
Epoch 45/50
7/7 [==============================] - 1s 124ms/step - loss:

7/7 [==============================] - 1s 132ms/step - loss: 0.0101 - acc: 0.9941
Epoch 23/50
7/7 [==============================] - 1s 133ms/step - loss: 0.0099 - acc: 0.9934
Epoch 24/50
7/7 [==============================] - 1s 127ms/step - loss: 0.0100 - acc: 0.9937
Epoch 25/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0097 - acc: 0.9937
Epoch 26/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0104 - acc: 0.9946
Epoch 27/50
7/7 [==============================] - 1s 132ms/step - loss: 0.0102 - acc: 0.9937
Epoch 28/50
7/7 [==============================] - 1s 125ms/step - loss: 0.0104 - acc: 0.9941
Epoch 29/50
7/7 [==============================] - 1s 133ms/step - loss: 0.0102 - acc: 0.9942
Epoch 30/50
7/7 [==============================] - 1s 136ms/step - loss: 0.0097 - acc: 0.9941
Epoch 31/50
7/7 [==============================] - 1s 132ms/step - loss: 0.0094 - acc: 0.9947
Epoch 32/50
7/7 [==============================] - 1s 125ms/step - loss:

7/7 [==============================] - 1s 128ms/step - loss: 0.0061 - acc: 0.9968
Epoch 10/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0062 - acc: 0.9968
Epoch 11/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0062 - acc: 0.9971
Epoch 12/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0063 - acc: 0.9970
Epoch 13/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0061 - acc: 0.9965
Epoch 14/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0056 - acc: 0.9968
Epoch 15/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0056 - acc: 0.9973
Epoch 16/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0056 - acc: 0.9970
Epoch 17/50
7/7 [==============================] - 1s 131ms/step - loss: 0.0054 - acc: 0.9973
Epoch 18/50
7/7 [==============================] - 1s 130ms/step - loss: 0.0055 - acc: 0.9968
Epoch 19/50
7/7 [==============================] - 1s 130ms/step - loss:

7/7 [==============================] - 1s 122ms/step - loss: 0.0044 - acc: 0.9979
Epoch 47/50
7/7 [==============================] - 1s 124ms/step - loss: 0.0044 - acc: 0.9974
Epoch 48/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0043 - acc: 0.9979
Epoch 49/50
7/7 [==============================] - 1s 123ms/step - loss: 0.0044 - acc: 0.9979
Epoch 50/50
7/7 [==============================] - 1s 123ms/step - loss: 0.0043 - acc: 0.9978
Epoch 1/50
7/7 [==============================] - 2s 130ms/step - loss: 0.0045 - acc: 0.9979
Epoch 2/50
7/7 [==============================] - 1s 133ms/step - loss: 0.0043 - acc: 0.9979
Epoch 3/50
7/7 [==============================] - 1s 125ms/step - loss: 0.0042 - acc: 0.9979
Epoch 4/50
7/7 [==============================] - 1s 130ms/step - loss: 0.0044 - acc: 0.9979
Epoch 5/50
7/7 [==============================] - 1s 129ms/step - loss: 0.0042 - acc: 0.9978
Epoch 6/50
7/7 [==============================] - 1s 124ms/step - loss: 0.004

...

7/7 [==============================] - 1s 132ms/step - loss: 0.0026 - acc: 0.9978
Epoch 24/50
7/7 [==============================] - 1s 135ms/step - loss: 0.0025 - acc: 0.9978
Epoch 25/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0025 - acc: 0.9978
Epoch 26/50
7/7 [==============================] - 1s 133ms/step - loss: 0.0025 - acc: 0.9978
Epoch 27/50
7/7 [==============================] - 1s 137ms/step - loss: 0.0025 - acc: 0.9978
Epoch 28/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0025 - acc: 0.9978
Epoch 29/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0025 - acc: 0.9978
Epoch 30/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0025 - acc: 0.9978
Epoch 31/50
7/7 [==============================] - 1s 127ms/step - loss: 0.0024 - acc: 0.9978
Epoch 32/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0024 - acc: 0.9978
Epoch 33/50
7/7 [==============================] - 1s 120ms/step - loss:

7/7 [==============================] - 1s 124ms/step - loss: 0.0022 - acc: 0.9981
Epoch 11/50
7/7 [==============================] - 1s 124ms/step - loss: 0.0022 - acc: 0.9981
Epoch 12/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0022 - acc: 0.9981
Epoch 13/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0022 - acc: 0.9981
Epoch 14/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0022 - acc: 0.9981
Epoch 15/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0022 - acc: 0.9981
Epoch 16/50
7/7 [==============================] - 1s 127ms/step - loss: 0.0022 - acc: 0.9981
Epoch 17/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0022 - acc: 0.9981
Epoch 18/50
7/7 [==============================] - 1s 125ms/step - loss: 0.0022 - acc: 0.9981
Epoch 19/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0021 - acc: 0.9981
Epoch 20/50
7/7 [==============================] - 1s 131ms/step - loss:

7/7 [==============================] - 1s 122ms/step - loss: 0.0022 - acc: 0.9981
Epoch 48/50
7/7 [==============================] - 1s 123ms/step - loss: 0.0021 - acc: 0.9981
Epoch 49/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0022 - acc: 0.9981
Epoch 50/50
7/7 [==============================] - 1s 121ms/step - loss: 0.0022 - acc: 0.9981
Epoch 1/50
7/7 [==============================] - 2s 128ms/step - loss: 0.0021 - acc: 0.9981
Epoch 2/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0022 - acc: 0.9981
Epoch 3/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0022 - acc: 0.9981
Epoch 4/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0021 - acc: 0.9981
Epoch 5/50
7/7 [==============================] - 1s 124ms/step - loss: 0.0022 - acc: 0.9981
Epoch 6/50
7/7 [==============================] - 1s 124ms/step - loss: 0.0022 - acc: 0.9981
Epoch 7/50
7/7 [==============================] - 1s 127ms/step - loss: 0.0021

7/7 [==============================] - 1s 120ms/step - loss: 0.0022 - acc: 0.9981
Epoch 35/50
7/7 [==============================] - 1s 121ms/step - loss: 0.0022 - acc: 0.9981
Epoch 36/50
7/7 [==============================] - 1s 123ms/step - loss: 0.0022 - acc: 0.9981
Epoch 37/50
7/7 [==============================] - 1s 121ms/step - loss: 0.0021 - acc: 0.9981
Epoch 38/50
7/7 [==============================] - 1s 128ms/step - loss: 0.0021 - acc: 0.9981
Epoch 39/50
7/7 [==============================] - 1s 123ms/step - loss: 0.0022 - acc: 0.9981
Epoch 40/50
7/7 [==============================] - 1s 118ms/step - loss: 0.0022 - acc: 0.9981
Epoch 41/50
7/7 [==============================] - 1s 122ms/step - loss: 0.0021 - acc: 0.9981
Epoch 42/50
7/7 [==============================] - 1s 121ms/step - loss: 0.0022 - acc: 0.9981
Epoch 43/50
7/7 [==============================] - 1s 123ms/step - loss: 0.0022 - acc: 0.9979
Epoch 44/50
7/7 [==============================] - 1s 121ms/step - loss:

7/7 [==============================] - 1s 131ms/step - loss: 0.0021 - acc: 0.9981
Epoch 22/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0021 - acc: 0.9981
Epoch 23/50
7/7 [==============================] - 1s 127ms/step - loss: 0.0021 - acc: 0.9981
Epoch 24/50
7/7 [==============================] - 1s 132ms/step - loss: 0.0021 - acc: 0.9981
Epoch 25/50
7/7 [==============================] - 1s 126ms/step - loss: 0.0021 - acc: 0.9981
Epoch 26/50
7/7 [==============================] - 1s 125ms/step - loss: 0.0021 - acc: 0.9981
Epoch 27/50
7/7 [==============================] - 1s 130ms/step - loss: 0.0022 - acc: 0.9981
Epoch 28/50
7/7 [==============================] - 1s 125ms/step - loss: 0.0022 - acc: 0.9981
Epoch 29/50
7/7 [==============================] - 1s 125ms/step - loss: 0.0022 - acc: 0.9981
Epoch 30/50
7/7 [==============================] - 1s 133ms/step - loss: 0.0022 - acc: 0.9981
Epoch 31/50
7/7 [==============================] - 1s 127ms/step - loss:

In [ ]:
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce(y_true, y_pred).numpy()

In [50]:
%%time
model2 = Sequential()
model2.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model2.add(LSTM(units=128, return_sequences=True))
model2.add(LSTM(units=64, return_sequences=True))
model2.add(LSTM(units=32, return_sequences=False))
# model2.add(LSTM(units=32))
model2.add(Dropout(0.2))
model2.add(Dense(units=1))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])
model2.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model2, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model2{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model2 = VotingClassifier(estimators = estimator, voting = 'soft')
final_model2.fit(X_train,y_train)

preds = final_model2.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 10, 128)           2149376   
_________________________________________________________________
lstm_8 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,211,233
Trainable params: 2,211,233
Non-trainable params: 0
_________________________________________________________________
[('model21', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fb8944a98b0>), ('model22', <keras.wrapp

25/25 [==============================] - 2s 76ms/step - loss: 0.5465 - acc: 0.8832
Epoch 12/20
25/25 [==============================] - 2s 75ms/step - loss: 0.3438 - acc: 0.9056
Epoch 13/20
25/25 [==============================] - 2s 75ms/step - loss: 0.2807 - acc: 0.9203
Epoch 14/20
25/25 [==============================] - 2s 75ms/step - loss: 0.2380 - acc: 0.9362
Epoch 15/20
25/25 [==============================] - 2s 76ms/step - loss: 0.2119 - acc: 0.9426
Epoch 16/20
25/25 [==============================] - 2s 75ms/step - loss: 0.1917 - acc: 0.9497
Epoch 17/20
25/25 [==============================] - 2s 75ms/step - loss: 0.1764 - acc: 0.9569
Epoch 18/20
25/25 [==============================] - 2s 75ms/step - loss: 0.1804 - acc: 0.9586
Epoch 19/20
25/25 [==============================] - 2s 75ms/step - loss: 0.1717 - acc: 0.9588
Epoch 20/20
25/25 [==============================] - 2s 74ms/step - loss: 0.1590 - acc: 0.9625
Epoch 1/20
25/25 [==============================] - 2s 75ms/st

25/25 [==============================] - 2s 74ms/step - loss: 0.2957 - acc: 0.9606
Epoch 18/20
25/25 [==============================] - 2s 75ms/step - loss: 1.5075 - acc: 0.8424
Epoch 19/20
25/25 [==============================] - 2s 75ms/step - loss: 0.7115 - acc: 0.8253
Epoch 20/20
25/25 [==============================] - 2s 74ms/step - loss: 0.2945 - acc: 0.9082
Epoch 1/20
25/25 [==============================] - 2s 76ms/step - loss: 0.4303 - acc: 0.9163
Epoch 2/20
25/25 [==============================] - 2s 76ms/step - loss: 0.3799 - acc: 0.8509
Epoch 3/20
25/25 [==============================] - 2s 75ms/step - loss: 0.2404 - acc: 0.9131
Epoch 4/20
25/25 [==============================] - 2s 74ms/step - loss: 0.2144 - acc: 0.9316
Epoch 5/20
25/25 [==============================] - 2s 73ms/step - loss: 0.4843 - acc: 0.7900
Epoch 6/20
25/25 [==============================] - 2s 76ms/step - loss: 0.2285 - acc: 0.9234
Epoch 7/20
25/25 [==============================] - 2s 76ms/step - l

25/25 [==============================] - 2s 75ms/step - loss: 0.0584 - acc: 0.9931
Epoch 4/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0575 - acc: 0.9925
Epoch 5/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0581 - acc: 0.9925
Epoch 6/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0585 - acc: 0.9929
Epoch 7/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0655 - acc: 0.9910
Epoch 8/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0626 - acc: 0.9921
Epoch 9/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0591 - acc: 0.9928
Epoch 10/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0594 - acc: 0.9909
Epoch 11/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0577 - acc: 0.9931
Epoch 12/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0596 - acc: 0.9928
Epoch 13/20
25/25 [==============================] - 2s 75ms/step - 

25/25 [==============================] - 2s 75ms/step - loss: 0.0954 - acc: 0.9769
Epoch 10/20
25/25 [==============================] - 2s 76ms/step - loss: 0.1200 - acc: 0.9678
Epoch 11/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0870 - acc: 0.9796
Epoch 12/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0786 - acc: 0.9827
Epoch 13/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0830 - acc: 0.9837
Epoch 14/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0799 - acc: 0.9841
Epoch 15/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0783 - acc: 0.9849
Epoch 16/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0730 - acc: 0.9840
Epoch 17/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0748 - acc: 0.9851
Epoch 18/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0897 - acc: 0.9804
Epoch 19/20
25/25 [==============================] - 2s 75ms/s

...

25/25 [==============================] - 2s 75ms/step - loss: 0.0471 - acc: 0.9954
Epoch 14/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0470 - acc: 0.9954
Epoch 15/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0468 - acc: 0.9954
Epoch 16/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0467 - acc: 0.9954
Epoch 17/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0471 - acc: 0.9954
Epoch 18/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0468 - acc: 0.9954
Epoch 19/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0468 - acc: 0.9954
Epoch 20/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0466 - acc: 0.9954
Epoch 1/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0470 - acc: 0.9954
Epoch 2/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0468 - acc: 0.9954
Epoch 3/20
25/25 [==============================] - 2s 74ms/step

25/25 [==============================] - 2s 74ms/step - loss: 0.0441 - acc: 0.9955
Epoch 20/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0443 - acc: 0.9955
Epoch 1/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0442 - acc: 0.9955
Epoch 2/20
25/25 [==============================] - 2s 77ms/step - loss: 0.0444 - acc: 0.9955
Epoch 3/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0444 - acc: 0.9955
Epoch 4/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0444 - acc: 0.9955
Epoch 5/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0442 - acc: 0.9955
Epoch 6/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0443 - acc: 0.9955
Epoch 7/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0444 - acc: 0.9955
Epoch 8/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0441 - acc: 0.9955
Epoch 9/20
25/25 [==============================] - 2s 75ms/step - los

25/25 [==============================] - 2s 74ms/step - loss: 0.0442 - acc: 0.9955
Epoch 6/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0440 - acc: 0.9955
Epoch 7/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0443 - acc: 0.9955
Epoch 8/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0440 - acc: 0.9955
Epoch 9/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0445 - acc: 0.9955
Epoch 10/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0443 - acc: 0.9955
Epoch 11/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0443 - acc: 0.9955
Epoch 12/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0445 - acc: 0.9955
Epoch 13/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0442 - acc: 0.9955
Epoch 14/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0442 - acc: 0.9955
Epoch 15/20
25/25 [==============================] - 2s 75ms/step 

25/25 [==============================] - 2s 76ms/step - loss: 0.0685 - acc: 0.9955
Epoch 12/20
25/25 [==============================] - 2s 77ms/step - loss: 0.0685 - acc: 0.9955
Epoch 13/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0685 - acc: 0.9955
Epoch 14/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0685 - acc: 0.9955
Epoch 15/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0685 - acc: 0.9955
Epoch 16/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0646 - acc: 0.9944
Epoch 17/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0562 - acc: 0.9952
Epoch 18/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0451 - acc: 0.9955
Epoch 19/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0446 - acc: 0.9955
Epoch 20/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0443 - acc: 0.9955
Epoch 1/20
25/25 [==============================] - 2s 75ms/st

25/25 [==============================] - 2s 75ms/step - loss: 0.0392 - acc: 0.9958
Epoch 18/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0392 - acc: 0.9958
Epoch 19/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0392 - acc: 0.9958
Epoch 20/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0394 - acc: 0.9958
Epoch 1/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0393 - acc: 0.9958
Epoch 2/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0390 - acc: 0.9958
Epoch 3/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0397 - acc: 0.9958
Epoch 4/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0390 - acc: 0.9958
Epoch 5/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0394 - acc: 0.9958
Epoch 6/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0395 - acc: 0.9958
Epoch 7/20
25/25 [==============================] - 2s 75ms/step - l

In [51]:
MSE(y_test, preds)

'2.66e-01'